In [1]:
import time
import sys
import demoji

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from tqdm import tqdm

# Start a Selenium driver
service = Service('./chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(service=service, options=options)

# Reach the deepL website
deepl_url = 'https://www.deepl.com/ko/translator' # 한국어
driver.get(deepl_url)

def remove_emojis(text):
    return demoji.replace(text, '')

In [2]:
# Define text to translate
for i in tqdm(range(5)):
    content = ''
    if i == 0:
        text_to_translate = 'Shuffle data after an epoch!'
    elif i == 1:
        text_to_translate = 'StaleElementReferenceException: Message: stale element reference: stale element not found'
    elif i == 2:
        text_to_translate = 'Hello python!'
    elif i == 3:
        text_to_translate = 'Using Selenium and deepL to automate the translation!'
    elif i == 4:
        text_to_translate = '😱 This is BMP!!!!'
    
    # Get thie inupt_area 
    #input_xpath = '//*[@id="panelTranslateText"]/div[1]/div[2]/section[1]/div[3]/div[2]/d-textarea/div/p'
    input_css = 'div.lmt__inner_textarea_container d-textarea'
    input_area = driver.find_element(By.CSS_SELECTOR, input_css)

    # Send the text
    try:
        input_area.send_keys(text_to_translate)
    except:
        # There is BMP(이모지). // remove
        text_to_translate = remove_emojis(text_to_translate)
        input_area.send_keys(text_to_translate)
        
    # Wait for translation to appear on the web page    
    time.sleep(3)
    
    bug_count = 0
    while (len(content) == 0) and (bug_count < 20):
        try:
            translation_xpath = '//*[@id="panelTranslateText"]/div[1]/div[2]/section[2]/div[3]/div[1]/d-textarea/div/p'
            translation_text = driver.find_element(By.XPATH, translation_xpath)
            content = translation_text.text
        except Exception as e:
            bug_count += 1
            
            if bug_count == 20:
                print(e)
                sys.exit()
    
    # Display results
    print('#'*50)
    print('Original    :', text_to_translate)
    print('Translation :', content)
    print('#'*50)
    
    # clear(X) button
    button_css = 'div.lmt__clear_text_button_wrapper button'
    button = driver.find_element(By.CSS_SELECTOR, button_css)
    button.click()

 20%|████████████████▊                                                                   | 1/5 [00:03<00:12,  3.17s/it]

##################################################
Original    : Shuffle data after an epoch!
Translation : 에포크 후 데이터 셔플!
##################################################


 40%|█████████████████████████████████▌                                                  | 2/5 [00:06<00:09,  3.24s/it]

##################################################
Original    : StaleElementReferenceException: Message: stale element reference: stale element not found
Translation : StaleElementReferenceException: 메시지: 부실 요소 참조: 부실 요소를 찾을 수 없습니다.
##################################################


 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:09<00:06,  3.20s/it]

##################################################
Original    : Hello python!
Translation : 안녕하세요 파이썬!
##################################################


 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:12<00:03,  3.23s/it]

##################################################
Original    : Using Selenium and deepL to automate the translation!
Translation : 셀레늄과 딥엘을 사용하여 번역을 자동화하세요!
##################################################


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:16<00:00,  3.24s/it]

##################################################
Original    :  This is BMP!!!!
Translation :  BMP!!!!
##################################################


In [3]:
# Quit selenium driver
driver.quit() 